In [1]:
from itertools import combinations, product

In [2]:
def make_chart(level):
    level_weight_mapper = {
        120: (7, 4, 360),
        130: (7, 4, 390),
        140: (8, 4, 420),
        150: (8, 4, 450),
        160: (9, 5, 480),
        200: (11, 6, 600),
    }

    weight1, weight2, weight3 = level_weight_mapper[level]
    stat1 = [weight1*i for i in range(3, 8)]
    stat2 = [weight2*i for i in range(3, 8)]
    stat3 = list(range(3, 8))
    stat4 = [weight3*i for i in range(3, 8)]

    chart = dict()
    for option in ['STR', 'DEX', 'LUK', 'INT', 'Armor']:
        chart[option] = stat1
    for option in ['STR+DEX', 'STR+LUK', 'STR+INT', 'DEX+LUK', 'DEX+INT', 'LUK+INT']:
        chart[option] = stat2
    for option in ['All', 'Attack', 'Magic', 'Speed', 'Jump']:
        chart[option] = stat3
    for option in ['HP', 'MP']:
        chart[option] = stat4
    
    chart['Level'] = [15, 20, 25, 30, 35]

    return chart

In [3]:
level = 200
chart = make_chart(level)

available_options = ['STR', 'STR+DEX', 'STR+LUK', 'STR+INT', 'All', 'Attack']
all_cases = list(combinations(chart.keys(), 4))  # 3,876 cases

prob_chart = {
    'strong': [0.2, 0.3, 0.36, 0.14, 0.],
    'forever': [0., 0.29, 0.45, 0.25, 0.01],
}

In [4]:
def all_cases_given_case(case, kind='forever'):
    case_available = [option for option in case if option in available_options]
    option_count = len(case_available)
    if option_count==0:
        return [[0, 0, 0, 1]]
    result = []; result_append = result.append  # (str, all, attack)

    if kind=='strong':
        grade_cases = product(*[range(4)]*option_count)
    elif kind=='forever':
        grade_cases = product(*[range(1, 5)]*option_count)
    probs = prob_chart[kind]

    for grades in grade_cases:
        temp_result = dict(zip(available_options, [0, 0, 0, 0, 0, 0]))
        prob = 1
        for grade in grades:
            prob *= probs[grade]

        for index, option in enumerate(case_available):
            temp_result[option] = chart[option][grades[index]]
        
        temp_result = list(temp_result.values())
        result_append([sum(temp_result[:-2]), *temp_result[-2:], prob])  # (str, all, attack, prob)
    
    return result

In [5]:
def option_prob_analysis(object_value, all_value=10.0, attack_value=4.0, kind='forever'):
    total_prob = 1
    stat_calculator = lambda str, all, attack: str + all*all_value + attack*attack_value
    results = []; results_extend = results.extend
    for case in all_cases:
        temp_results = all_cases_given_case(case, kind=kind)
        for (str, all, attack, prob) in temp_results:
            if stat_calculator(str, all, attack) >= object_value:
                total_prob += prob
    
    return total_prob / 3876

In [6]:
object_value = 140
all_value = 10.0
attack_value = 4.0
kind_name = '영환불'
kind_mapper = {'강환불': 'strong', '영환불': 'forever'}

option_prob = option_prob_analysis(object_value, all_value, attack_value, kind_mapper[kind_name])
expectation = 1/option_prob
print(f'{object_value:3d} 이상의 추옵을 {kind_name}로 뽑을 확률은   {option_prob:6.1%}  입니다.')
print(f'{object_value:3d} 이상의 추옵을 {kind_name}로 뽑는 기대값은 {expectation:6.1f}개입니다.')

140 이상의 추옵을 영환불로 뽑을 확률은     0.6%  입니다.
140 이상의 추옵을 영환불로 뽑는 기대값은  154.2개입니다.
